<a href="https://colab.research.google.com/github/LuViBeBe93/tesis/blob/main/T5_F5_op.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers pandas torch scikit-learn bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 993.8 kB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
import math
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score as bert_score
from torch.utils.data import DataLoader

In [3]:
# Hiperparámetros
HYPERPARAMS = {
    "model_name": "t5-small",
    "max_length": 128,
    "batch_size": 8, #Este parámetro sale de la optimización bayesiana con Optuna
    "num_epochs": 14, #Este parámetro sale de la optimización bayesiana con Optuna
    "learning_rate": 5.28e-5, #Este parámetro sale de la optimización bayesiana con Optuna
    "num_beams": 4, #Este parámetro sale de la optimización bayesiana con Optuna
    "no_repeat_ngram_size": 3, #Este parámetro sale de la optimización bayesiana con Optuna
    "early_stopping": True,
    "logging_steps": 100,
    "output_dir": "./noticias_t5_model",
    "logging_dir": "./logs",
    "save_steps": 500,
    "save_total_limit": 2
}

In [4]:
# Cargar datos
from google.colab import files
uploaded = files.upload()

data = pd.read_excel('BASE_30122024.xlsx')
data = pd.DataFrame(data)

Saving BASE_30122024.xlsx to BASE_30122024.xlsx


In [5]:
# Convertir datos numéricos a texto
def datos_a_texto(row):
    texto_datos = (
        f"Año: {row.get('Año', 'NaN')}, Mes: {row.get('Mes', 'NaN')}, "
        f"Trimestre: {row.get('Trimestre', 'NaN')}, Frecuencia: {row.get('Frecuencia', 'NaN')}, "
        f"Año_comparación: {row.get('Año_comparación', 'NaN')}, País: {row.get('País', 'NaN')}, "
        f"Tendencia: {row.get('Tendencia', 'NaN')}, Sector: {row.get('Sector', 'NaN')}, "
        f"Indicador: {row.get('Indicador', 'NaN')}, Valor_actual: {row.get('Valor_actual', 'NaN')}, "
        f"Variación: {row.get('Variación', 'NaN')}, Valor_comparación: {row.get('Valor_comparación', 'NaN')}, "
        f"Entidad: {row.get('Entidad', 'NaN')}"
    )
    return texto_datos

data["entrada_texto"] = data.apply(datos_a_texto, axis=1)
data["texto_final"] = data["entrada_texto"] + " </s> Noticia: " + data["Noticia"]


In [6]:
# Clase Dataset personalizada
class NoticiasDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entrada_texto = self.data.iloc[idx]["entrada_texto"]
        noticia = self.data.iloc[idx]["Noticia"]

        input_encoding = self.tokenizer(
            entrada_texto,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            noticia,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "labels": target_encoding["input_ids"].flatten(),
        }

In [7]:
# Dividir datos en 80% entrenamiento y 20% evaluación
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)

In [8]:
# Cargar tokenizador y modelo preentrenado
tokenizer = T5Tokenizer.from_pretrained(HYPERPARAMS["model_name"])
model = T5ForConditionalGeneration.from_pretrained(HYPERPARAMS["model_name"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
# Crear datasets
train_dataset = NoticiasDataset(train_data, tokenizer, HYPERPARAMS["max_length"])
eval_dataset = NoticiasDataset(eval_data, tokenizer, HYPERPARAMS["max_length"])

In [10]:
# Crear DataLoader para los conjuntos de datos
train_dataloader = DataLoader(train_dataset, batch_size=HYPERPARAMS["batch_size"], shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=HYPERPARAMS["batch_size"], shuffle=False)

In [11]:
# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir=HYPERPARAMS["output_dir"],
    num_train_epochs=HYPERPARAMS["num_epochs"],
    per_device_train_batch_size=HYPERPARAMS["batch_size"],
    save_steps=HYPERPARAMS["save_steps"],
    save_total_limit=HYPERPARAMS["save_total_limit"],
    logging_dir=HYPERPARAMS["logging_dir"],
    logging_steps=HYPERPARAMS["logging_steps"],
    learning_rate=HYPERPARAMS["learning_rate"],
)

# Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Entrenar modelo
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,2.338000
200,0.753500
300,0.632500
400,0.545300
500,0.494500
600,0.442600
700,0.417500
800,0.406500
900,0.360300
1000,0.368100


TrainOutput(global_step=4466, training_loss=0.32787624090923306, metrics={'train_runtime': 843.93, 'train_samples_per_second': 42.302, 'train_steps_per_second': 5.292, 'total_flos': 1207925578137600.0, 'train_loss': 0.32787624090923306, 'epoch': 14.0})

In [12]:
# Generar noticias con parámetros adicionales
def generar_noticia(model, tokenizer, datos):
    # Convertir los datos a texto
    entrada_texto = datos_a_texto(datos)

    # Codificar la entrada
    inputs = tokenizer.encode(entrada_texto, return_tensors="pt")

    # Mover los tensores al mismo dispositivo que el modelo
    inputs = inputs.to(model.device)  # Mover los inputs a la misma device que el modelo

    # Generar la salida con los parámetros dados
    outputs = model.generate(
        inputs,
        max_length=HYPERPARAMS["max_length"],
        num_beams=HYPERPARAMS["num_beams"],
        no_repeat_ngram_size=HYPERPARAMS["no_repeat_ngram_size"],
        early_stopping=HYPERPARAMS["early_stopping"],
        temperature=0.5,  #Este parámetro sale de la optimización por grilla
        top_p=0.6,  #Este parámetro sale de la optimización por grilla
        do_sample=True  # Activa la aleatorización (en lugar de beam search)
    )

    # Decodificar la salida
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [13]:
# Métricas de evaluación
def calcular_perplejidad(model, tokenizer, eval_dataloader):
    model.eval()  # Establecer el modelo en modo evaluación
    total_loss = 0
    total_tokens = 0

    # Asegurarse de que los datos estén en el mismo dispositivo que el modelo
    for batch in eval_dataloader:
        input_ids = batch['input_ids'].to(model.device)  # Mover a GPU/CPU según corresponda
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        # Realizar una pasada de evaluación sin retropropagación
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            # La salida debería incluir `loss` y posiblemente `logits`
            if isinstance(outputs, tuple):
                loss = outputs[0]  # La primera parte es la pérdida
            else:
                loss = outputs.loss  # Si no es un tuple, el objeto debe tener `loss`

            total_loss += loss.item() * input_ids.size(0)  # Multiplicar por el tamaño del batch
            total_tokens += input_ids.size(0)

    # Calcular la perplejidad
    perplejidad = math.exp(total_loss / total_tokens)
    return perplejidad


# Función para calcular BLEU score
def calcular_bleu_score(model, tokenizer, eval_dataset):
    model.eval()
    bleu_scores = []

    for batch in eval_dataset:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        # Realizar una pasada de evaluación sin retropropagación
        with torch.no_grad():
            outputs = model.generate(input_ids, attention_mask=attention_mask)
            decoded_preds = tokenizer.decode(outputs[0], skip_special_tokens=True)
            decoded_labels = tokenizer.decode(labels[0], skip_special_tokens=True)

            # Calcular el BLEU score para esta predicción
            bleu_scores.append(sentence_bleu([decoded_labels.split()], decoded_preds.split()))

    # Promediar el BLEU score
    return np.mean(bleu_scores)


def calcular_bertscore(model, tokenizer, dataset):
    model.eval()
    referencias = []
    generadas = []

    for i in range(len(dataset)):
        datos = eval_data.iloc[i]
        referencias.append(datos["Noticia"])
        generadas.append(generar_noticia(model, tokenizer, datos))

    P, R, F1 = bert_score(generadas, referencias, lang="es", model_type="bert-base-uncased")
    return {
        "Precision promedio": P.mean().item(),
        "Recall promedio": R.mean().item(),
        "F1 promedio": F1.mean().item(),
    }


In [14]:
# Evaluar modelo en el 20% de datos
perplejidad = calcular_perplejidad(model, tokenizer, eval_dataloader)
print(f"Perplejidad en conjunto de evaluación: {perplejidad:.2f}")

bleu_score = calcular_bleu_score(model, tokenizer, eval_dataloader)
print(f"BLEU Score promedio en conjunto de evaluación: {bleu_score:.2f}")

bertscore_resultados = calcular_bertscore(model, tokenizer, eval_dataloader)
print("Resultados de BERTScore en conjunto de evaluación:")
print(f"  Precision promedio: {bertscore_resultados['Precision promedio']:.2f}")
print(f"  Recall promedio: {bertscore_resultados['Recall promedio']:.2f}")
print(f"  F1 promedio: {bertscore_resultados['F1 promedio']:.2f}")

Perplejidad en conjunto de evaluación: 1.11


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

BLEU Score promedio en conjunto de evaluación: 0.14


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Resultados de BERTScore en conjunto de evaluación:
  Precision promedio: 0.84
  Recall promedio: 0.82
  F1 promedio: 0.83
